In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val

In [2]:
DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + '/data/tuner'
CROSSVAL_DIR = PWD + '/data/cross_val'

PLANE = 2
CHANNEL = 11

PROJECT_NAME = f'mlp_{PLANE}_{CHANNEL}'

LR = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=6, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.1, 0.2])

    model = keras.Sequential()
    model.add(layers.Input(24))
    if hp_input_batch_normalization:
        model.add(layers.BatchNormalization())

    n_units = 3 * (2 ** (hp_n_hidden_layers - 1)) * hp_units_mult
    for _ in range(hp_n_hidden_layers):
        model.add(layers.Dense(n_units, activation='relu'))
        n_units //= 2
        if hp_batch_normalization:
            model.add(layers.BatchNormalization())
        if hp_dropout > 0:
            model.add(layers.Dropout(hp_dropout))

    model.add(layers.Dense(1))

    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)

    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                600       
                                                                 
 dense_1 (Dense)             (None, 12)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10),
    callbacks.TensorBoard("/tmp/tb_logs/bayesian/" + PROJECT_NAME)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR + "/bayesian",
                                         project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_base_train, y_base_train, validation_split=0.2, epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 31s]
val_loss: 8.98246145248413

Best val_loss So Far: 8.571378707885742
Total elapsed time: 00h 21m 12s


# Results

In [8]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 1 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 4
units_mult: 8
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.571378707885742


In [9]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 24)               96        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 192)               4800      
                                                                 
 batch_normalization_1 (Batc  (None, 192)              768       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 192)               0         
                                                                 
 dense_1 (Dense)             (None, 96)                18528     
                                                                 
 batch_normalization_2 (Batc  (None, 96)               3

---

In [10]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 4, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 29953
========== Model 1 ==========
{'n_hidden_layers': 3, 'units_mult': 4, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 2737
========== Model 2 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2545
========== Model 3 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2737
========== Model 4 ==========
{'n_hidden_layers': 5, 'units_mult': 16, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 411697


In [11]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/tuner/bayesian\mlp_2_11
Showing 5 best trials
Trial summary
Hyperparameters:
n_hidden_layers: 4
units_mult: 8
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.571378707885742
Trial summary
Hyperparameters:
n_hidden_layers: 3
units_mult: 4
batch_normalization: False
input_batch_normalization: True
dropout: 0.0
Score: 8.589727401733398
Trial summary
Hyperparameters:
n_hidden_layers: 1
units_mult: 32
batch_normalization: False
input_batch_normalization: True
dropout: 0.2
Score: 8.590714931488037
Trial summary
Hyperparameters:
n_hidden_layers: 1
units_mult: 32
batch_normalization: True
input_batch_normalization: True
dropout: 0.2
Score: 8.593961715698242
Trial summary
Hyperparameters:
n_hidden_layers: 5
units_mult: 16
batch_normalization: False
input_batch_normalization: True
dropout: 0.0
Score: 8.60997486114502


# Cross-validation for top 5 models

In [12]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=False)
model_scores = cross_validator()

{'n_hidden_layers': 4, 'units_mult': 8, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 29953
Got score: 8.6960 (8.6609, 8.7311)
Got score: 8.3512 (8.3741, 8.3284)
Got score: 8.1229 (8.1064, 8.1393)
Got score: 8.4067 (8.3944, 8.4191)
Got score: 8.7155 (8.6960, 8.7351)


{'n_hidden_layers': 3, 'units_mult': 4, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 2737
Got score: 8.8446 (8.8814, 8.8078)
Got score: 8.6175 (8.5951, 8.6399)
Got score: 8.2877 (8.3585, 8.2168)
Got score: 8.5512 (8.6143, 8.4881)
Got score: 8.8314 (8.8635, 8.7993)


{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2545
Got score: 8.8180 (8.6709, 8.9651)
Got score: 8.3381 (8.3566, 8.3197)
Got score: 8.1193 (8.1428, 8.0958)
Got score: 8.4218 (8.4380, 8.4055)
Got score: 8.7505 (8.7030, 8.7980)


{'n_hidden_layers': 1, 'units_mult': 32, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 2737
Got score: 8.7369 (8.6635, 8.8102)
Got score: 8.5558 (8.6334, 8.4782)
Got score: 8.3401 (8.4918, 8.1884)
Got score: 8.5635 (8.5709, 8.5561)
Got score: 8.8504 (8.8071, 8.8936)


{'n_hidden_layers': 5, 'units_mult': 16, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 411697
Got score: 8.7897 (8.8513, 8.7281)
Got score: 8.3864 (8.3457, 8.4270)
Got score: 8.3028 (8.3325, 8.2731)
Got score: 8.5401 (8.6465, 8.4337)
Got score: 8.9012 (8.9223, 8.8801)


In [13]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.46,0.22,29953
1,8.63,0.20,2737
2,8.49,0.26,2545
3,8.61,0.17,2737
4,8.58,0.23,411697
